In [39]:
import pandas as pd
from tabulate import tabulate

In [40]:
df = pd.read_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F1D.csv')
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

+------+-------+------------------------+---------+--------------------+--------------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|      | frame |          bbox          | face_id |    detect_conf     |     track_conf     |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    |
+------+-------+------------------------+---------+--------------------+--------------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|  0   |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |        nan         |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |   0.96975315    |    4.3312063    |       m       | 38.312063217163086 |
|  1   |   0   |  (521, 121, 555, 155)  |    1    | 0.9997656345367432 |        

In [41]:
df = df.drop_duplicates(subset=['face_id'], keep='first')
df.reset_index(drop=True, inplace=True)
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

+------+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|      | frame |          bbox          | face_id |    detect_conf     | track_conf |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    |
+------+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|  0   |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |    nan     |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |   0.96975315    |    4.3312063    |       m       | 38.312063217163086 |
|  1   |   0   |  (521, 121, 555, 155)  |    1    | 0.9997656345367432 |    nan     |  0.29838723   |  3.4812188

In [43]:
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # area de inters
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # area de 2 bbox
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # area de uniao
    unionArea = boxAArea + boxBArea - interArea
    
    iou = interArea / float(unionArea)
    return iou

In [44]:

iou_threshold = 0.3

# coluna para verificar os ids de face q sao iguais
df['same_face_id'] = -1
ious = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        boxA = eval(df.loc[i, 'bbox'])
        boxB = eval(df.loc[j, 'bbox'])
        iou = calculate_iou(boxA, boxB)
        ious.append(iou)
        if iou > iou_threshold:
            if df.loc[i, 'same_face_id'] == -1 and df.loc[j, 'same_face_id'] == -1:
                df.loc[i, 'same_face_id'] = df.loc[i, 'face_id']
                df.loc[j, 'same_face_id'] = df.loc[i, 'face_id']
            elif df.loc[i, 'same_face_id'] != -1:
                df.loc[j, 'same_face_id'] = df.loc[i, 'same_face_id']
            elif df.loc[j, 'same_face_id'] != -1:
                df.loc[i, 'same_face_id'] = df.loc[j, 'same_face_id']

# se ainda for -1, nao teve nenhuma correspodencia, entao setamos com o proprio id.
df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']



In [45]:
print(tabulate(df, headers='keys', tablefmt='pretty'))
# print(ious)


# filtered_df = df[df['same_face_id'] == 0]
# print(tabulate(filtered_df, headers='keys', tablefmt='pretty'))

+------+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+--------------+
|      | frame |          bbox          | face_id |    detect_conf     | track_conf |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    | same_face_id |
+------+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+--------------+
|  0   |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |    nan     |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |   0.96975315    |    4.3312063    |       m       | 38.312063217163086 |      0       |
|  1   |   0   |  (521, 121, 555, 155)  |    1    | 

In [31]:
grouped_df = df.groupby('same_face_id').agg({
    'sex_decfunc_avg': 'mean',  # media do sex_decfunc_avg
    'age_label_avg': 'mean'   # media do age_decfunc_avg
}).reset_index()

print(tabulate(grouped_df, headers='keys', tablefmt='pretty'))

+----+--------------+----------------------+--------------------+
|    | same_face_id |   sex_decfunc_avg    |   age_label_avg    |
+----+--------------+----------------------+--------------------+
| 0  |     0.0      | 0.32131358427226275  | 25.963231081510113 |
| 1  |     1.0      |      -2.0359038      | 31.649131178855896 |
| 2  |     2.0      |     0.436494638      | 24.252681493759155 |
| 3  |     3.0      |     -2.57119835      | 32.64615058898926  |
| 4  |     5.0      | -1.0254403397826086  | 25.44952579166578  |
| 5  |     24.0     |     1.298206135      | 23.51027250289917  |
| 6  |     28.0     |      1.1099344       | 27.386164665222168 |
| 7  |     34.0     |  0.9549141064285714  | 22.55213975906372  |
| 8  |     35.0     |     1.849308444      | 28.12223505973816  |
| 9  |     43.0     | -0.24621632413636363 | 24.82390132817355  |
| 10 |     44.0     |      -2.5348582      | 30.827877521514893 |
| 11 |     46.0     | -0.9766324714444443  | 29.93601719538371  |
| 12 |    